In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
output_notebook()
import numpy as np
import pandas as pd
from main import Atmosphere, justdoit
import astropy.units as u

Loading BokehJS ...

In [15]:
a = Atmosphere(['Cr','MgSiO3'])

a.get_gravity(gravity=357.00, gravity_unit=u.Unit('cm/(s**2)'))
a.get_pt( filename='../temp.dat',skiprows=3, delim_whitespace=True, header=None, names=["ind","pressure","temperature","kzz"])
a.get_kz()
opd, w0, g0, opd_gas = justdoit(a, directory='/Users/natashabatalha/Documents/eddysed/input/optics/')

In [16]:
df = pd.DataFrame(index=[ i for i in range(opd.shape[0]*opd.shape[1])], columns=['lvl','w','opd','w0','g0'])
i = 0 
LVL = []
WV,OPD,WW0,GG0 =[],[],[],[]
for j in range(opd.shape[0]):
       for w in range(opd.shape[1]):
            LVL +=[j+1]
            WV+=[w+1]
            OPD+=[opd[j,w]]
            WW0+=[w0[j,w]]
            GG0+=[g0[j,w]]
df.iloc[:,0 ] = LVL
df.iloc[:,1 ] = WV
df.iloc[:,2 ] = OPD
df.iloc[:,3 ] = WW0
df.iloc[:,4 ] = GG0

In [17]:
import picaso.justplotit as jpi

In [18]:
am = pd.read_csv('../inputTASH.cld', delim_whitespace=True)
am.head()

,lvl,w,pressure,opd,g0,w0,sigma
0,1,1,2.9176,6.5334,-0.008979,4.000000e-07,8.404700e-21
1,1,2,2.9176,17.5430,-0.002606,1.000000e-06,2.256800e-20
2,1,3,2.9176,33.0420,0.000656,2.200000e-06,4.250500e-20
3,1,4,2.9176,58.4640,0.000990,3.800000e-06,7.521000e-20
4,1,5,2.9176,110.2300,0.000402,7.200000e-06,1.418100e-19


In [19]:
show(jpi.plot_cld_input(nwno=196, nlayer=52, filename='../inputTASH.cld',delim_whitespace=True))
show(jpi.plot_cld_input(nwno=196, nlayer=52,df=df))

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column, row
import pandas as pd

In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
labels =['scat','ext','cqs'] #['qc_layer', 'qt_layer', 'rg_layer', 'ndz_layer']
#y_axis_type='log',y_range=[1e2,1e-6],
a = [figure(y_axis_type='log', x_axis_type='log',y_axis_label=labels[i], height=300) for i in range(3)]
py = pd.read_csv('../py.txt',delim_whitespace=True, header=None)
fort = pd.read_csv('../fort.txt',delim_whitespace=True, header=None)

for i in range(2):
    a[i].circle(fort[0],py[i+1],color='red')
    a[i].line(fort[0],abs(py[i+1]),color='red')
    a[i].circle(fort[0],fort[i+1],color='black',legend='me')
    a[i].line(fort[0],abs(fort[i+1]),color='black',legend='me')
show(column([i for i in a]))

In [10]:
py[]

SyntaxError: invalid syntax (<ipython-input-10-647ba2d50440>, line 1)